#### 이미지 전처리
이미지를 그레이스케일로 변환하고 블러를 적용한 뒤, Canny Edge Detection 등을 사용하여 윤곽선을 감지합니다. 이는 영수증의 경계를 강조하는 데 도움이 됩니다.


In [ ]:
import cv2

# 이미지 불러오기
image = cv2.imread('receipt.jpg')

# 그레이스케일 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 블러 적용 (노이즈 제거)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Canny Edge Detection
edges = cv2.Canny(blurred, 50, 150)

#### 윤곽선 찾기
감지된 윤곽선 중 사각형 모양의 가장 큰 윤곽선을 찾아 영수증 영역으로 간주합니다. OpenCV의 findContours와 approxPolyDP 함수를 사용하여 윤곽선을 찾고 단순화할 수 있습니다.

In [ ]:
# 윤곽선 찾기
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 윤곽선 찾기
receipt_contour = max(contours, key=cv2.contourArea)

# 윤곽선을 사각형으로 근사화
epsilon = 0.02 * cv2.arcLength(receipt_contour, True)
approx = cv2.approxPolyDP(receipt_contour, epsilon, True)

#### 꼭짓점 좌표 추출
영수증의 윤곽선을 근사화한 후, 사각형 모양이라면 4개의 꼭짓점 좌표를 얻게 됩니다. 이 좌표들을 Perspective Transform의 입력 좌표로 사용할 수 있습니다.

In [ ]:
# 사각형 좌표 확인
if len(approx) == 4:
    pts = approx.reshape(4, 2)  # 4개의 꼭짓점 좌표
else:
    print("영수증 윤곽을 사각형으로 감지하지 못했습니다.")

#### 좌표 순서 정렬
pts에 있는 좌표를 적절한 순서(좌상단, 우상단, 좌하단, 우하단)로 정렬하여 원근 변환에 사용할 수 있도록 합니다. 일반적으로 x, y 좌표의 합과 차를 기준으로 정렬합니다.

In [ ]:
import numpy as np

# 좌표를 순서대로 정렬
rect = np.zeros((4, 2), dtype="float32")
s = pts.sum(axis=1)
rect[0] = pts[np.argmin(s)]  # 좌상단
rect[2] = pts[np.argmax(s)]  # 우하단

diff = np.diff(pts, axis=1)
rect[1] = pts[np.argmin(diff)]  # 우상단
rect[3] = pts[np.argmax(diff)]  # 좌하단


#### 결과 확인 및 원근 변환
이렇게 추출된 rect 좌표를 사용하여 영수증 영역을 잘라내거나 Perspective Augmentation을 적용할 수 있습니다.

In [ ]:
# 변환 후 이미지 크기 설정
width, height = 500, 300
dst = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

# 원근 변환 행렬 계산 및 적용
matrix = cv2.getPerspectiveTransform(rect, dst)
warped = cv2.warpPerspective(image, matrix, (width, height))

# 결과 이미지 저장
cv2.imwrite('warped_receipt.jpg', warped)

이 방법을 통해 이미지에서 영수증의 좌표를 자동으로 추출할 수 있으며, 다양한 각도와 시점에서도 영수증을 정확히 인식하는 데 도움이 됩니다. 만약 윤곽선이 잘 감지되지 않는다면, 이미지 전처리 단계를 조정하거나 더 강력한 경계선 감지 방법을 적용해 볼 수 있습니다.